In [27]:
import pandas as pd
import requests
from time import time
from calendar import monthrange
import re
import os

from IPython.display import clear_output

from multiprocessing import Pool

### Build request list

In [28]:
def get_req(i):
    d = str(i.day).zfill(2)
    m = str(i.month).zfill(2)
    y = str(i.year)
    h = str(i.hour).zfill(2)
    req = f'https://www.ogimet.com/cgi-bin/gsynres?lang=en&osum=no&state=Serb&fmt=html&ano={y}&mes={m}&day={d}&hora={h}&ord=REV'
    return req

def f(val):
    try:
        return float(val)
    except:
        return None

In [29]:
workers = Pool()

date_range = pd.date_range('01-01-2000 00:00:00', '01-01-2020 00:00:00', freq='H', closed='left')
%time req_list = workers.map(get_req, date_range)

workers.close()

CPU times: user 910 ms, sys: 225 ms, total: 1.14 s
Wall time: 1.1 s


In [33]:
pd.read_html(req_list[0])[0]

Station  \
                                             Station   
0  Professional information about meteorological ...   
1  OGIMET Versión española METEOSAT Meteosat-9/me...   
2  DISCLAIMER: The information in these pages mus...   
3                                            Summary   

                                      Temperature(C)  \
                                                 Max   
0  (adsbygoogle = window.adsbygoogle || []).push(...   
1  Synop summary for 26 stations in Serbia OGIMET...   
2  DISCLAIMER: The information in these pages mus...   
3                                                0.1   

                                                          Td.Med(C) Hr.Med(%)  \
                                                 Min  Med Td.Med(C) Hr.Med(%)   
0  (adsbygoogle = window.adsbygoogle || []).push(...  NaN       NaN       NaN   
1  (adsbygoogle = window.adsbygoogle || []).push(...  NaN       NaN       NaN   
2  DISCLAIMER: The information in these pages mus...  NaN       NaN       NaN   
3                                               -3.0 -1.5      -2.5      93.1   

  Wind(km/h)            Pres.s.lev(Hp)  ... VisKm SnowDep.(cm)  \
        Dir. Int.  Gust Pres.s.lev(Hp)  ... VisKm SnowDep.(cm)   
0        NaN  NaN   NaN            NaN  ...   NaN          NaN   
1        NaN  NaN   NaN            NaN  ...   NaN          NaN   
2        NaN  NaN   NaN            NaN  ...   NaN          NaN   
3        NNW  7.7  72.0         1022.8  ...   5.3         14.0   

  Dailyweather summary                                                \
  Dailyweather summary Dailyweather summary.1 Dailyweather summary.2   
0                  NaN                    NaN                    NaN   
1                  NaN                    NaN                    NaN   
2                  NaN                    NaN                    NaN   
3                  NaN                    NaN                    NaN   

                                                                        \
  Dailyweather summary.3 Dailyweather summary.4 Dailyweather summary.5   
0                    NaN                    NaN                    NaN   
1                    NaN                    NaN                    NaN   
2                    NaN                    NaN                    NaN   
3                    NaN                    NaN                    NaN   

                                                 
  Dailyweather summary.6 Dailyweather summary.7  
0                    NaN                    NaN  
1                    NaN                    NaN  
2                    NaN                    NaN  
3                    NaN                    NaN  

[4 rows x 24 columns]

### Download hourly csvs

In [4]:
def download_csv(req):
    f_name = '_'.join(re.findall('ano=(.*)&mes=(.*)&day=(.*)&hora=(.*)&', req)[0])+'.gz'
    path = '../../data/serbia/csv/'+f_name
    if not os.path.exists(path):
        try:
            res = requests.get(req)
            pd.read_html(res.text)[2].to_csv(path, index=None, compression='gzip')
        except:
            pd.to_pickle(f_name, path.replace('.gz','.log'))

In [5]:
workers = Pool()

%time workers.map(download_csv, req_list)

workers.close()

CPU times: user 76.8 ms, sys: 70.5 ms, total: 147 ms
Wall time: 4.1 s


### Download monthly csvs

In [3]:
def deg_to_dec(x):
    try:
        a, b = map(int, x.split('-'))
        return a+b/60
    except:
        a, b, c = map(int, x.split('-'))
        return a+b/60+c/3600

In [4]:
stations = [[13067, 'palic', '46-06', '19-46'],
            [13160, 'sombor', '45-46', '19-09'],
            [13168, 'novi sad rimski sancevi', '45-20', '19-51'], 
            [13173, 'zrenjanin', '45-22', '20-25'], 
            [13174, 'kikinda', '45-51', '20-28'],
            [13180, 'banatski karlovac', '45-03', '21-02'],
            [13183, 'vrsac', '45-09', '21-19'],
            [13262, 'loznica', '44-33', '19-14'],
            [13266, 'sremska mitrovica', '45-06', '19-33'],
            [13269, 'valjevo', '44-19', '19-55'],
            [13272, 'beograd / surcin', '44-49-28', '20-17-28'],
            [13274, 'beograd', '44-48', '20-28'],
            [13279, 'smederevska palanka', '44-22', '20-57'],
            [13285, 'veliko gradiste', '44-45', '21-31'],
            [13289, 'crni vrh', '44-07', '21-57'],
            [13295, 'negotin', '44-14', '22-33'],
            [13367, 'zlatibor', '43-44', '19-43'],
            [13369, 'sjenica', '43-17', '20-00'],
            [13370, 'pozega uzicka', '43-50', '20-02'],
            [13376, 'kraljevo', '43-42', '20-42'],
            [13378, 'kopaonik', '43-17', '20-48'],
            [13383, 'krusevac', '43-34', '21-21'],
            [13384, 'cuprija', '43-56', '21-23'],
            [13388, 'nis', '43-20', '21-54'],
            [13389, 'leskovac', '42-59', '21-57'],
            [13397, 'dimitrovgrad', '43-01', '22-45'],
            [13489, 'vranje', '42-33', '21-55']]

station_info = pd.DataFrame(stations, columns=['code', 'name', 'latitude', 'longitude'])
station_info['latitude'] = station_info['latitude'].apply(deg_to_dec)
station_info['longitude'] = station_info['longitude'].apply(deg_to_dec)
station_info

,code,name,latitude,longitude
0,13067,palic,46.100000,19.766667
1,13160,sombor,45.766667,19.150000
2,13168,novi sad rimski sancevi,45.333333,19.850000
3,13173,zrenjanin,45.366667,20.416667
4,13174,kikinda,45.850000,20.466667
5,13180,banatski karlovac,45.050000,21.033333
6,13183,vrsac,45.150000,21.316667
7,13262,loznica,44.550000,19.233333
8,13266,sremska mitrovica,45.100000,19.550000
9,13269,valjevo,44.316667,19.916667


In [5]:
def get_req(x):
    s, i = x
    y = i.year
    m = str(i.month).zfill(2)
    d = str(i.day).zfill(2)
    req = f'https://www.ogimet.com/cgi-bin/gsynres?lang=en&ind={s}&decoded=yes&ndays={30}&ano={y}&mes={m}&day={d}&hora=00'
    return req

def download_csv(req):
    f_name = '_'.join(re.findall('&ind=(.*)&decoded=yes&ndays=30&ano=(.*)&mes=(.*)&day=(.*)&', req)[0])+'.gz'
    path = '../../data/serbia/monthly/'+f_name
    if not os.path.exists(path):
        try:
            res = requests.get(req)
            raw_df = pd.read_html(res.text)[2]
            df = pd.DataFrame()
            cols = raw_df.columns
            flag = False
            for col in cols:
                if flag and col[0] == 'Date':
                    df['Time'] = col[1:]
                    continue
                if col[0] == 'Date':
                    df['Date'] = col[1:]
                    flag=True
                df[col[0]] = col[1:]
            df.to_csv(path, index=None, compression='gzip')
        except:
            pd.to_pickle(f_name, path.replace('.gz','.log'))

In [6]:
workers = Pool()

date_range = pd.date_range('01-01-2000', '31-01-2021', freq='30D')
main_req_list = []
for s in station_info['code']:
    s_date_range = [(s, i) for i in date_range]
    req_list = workers.map(get_req, s_date_range)
    main_req_list.extend(req_list)

workers.close()
len(main_req_list)

6939

In [11]:
workers = Pool()

%time workers.map(download_csv, main_req_list)

workers.close()

CPU times: user 233 ms, sys: 116 ms, total: 349 ms
Wall time: 34min 41s


In [14]:
pd.read_html(main_req_list[0])[0]

Date  \
                                          01/01/2000   
                                          12/31/1999   
                                          12/31/1999   
                                          12/31/1999   
                                          12/31/1999   
                                          12/31/1999   
                                          12/31/1999   
                                          12/31/1999   
                                          12/30/1999   
                                          12/30/1999   
                                          12/30/1999   
                                          12/30/1999   
                                          12/30/1999   
                                          12/30/1999   
                                          12/30/1999   
                                          12/30/1999   
                                          12/29/1999   
                                          12/29/1999   
                                          12/29/1999   
                                          12/29/1999   
                                          12/29/1999   
                                          12/29/1999   
                                          12/29/1999   
                                          12/29/1999   
                                          12/28/1999   
                                          12/28/1999   
                                          12/28/1999   
                                          12/28/1999   
                                          12/28/1999   
                                          12/28/1999   
                                          12/28/1999   
                                          12/28/1999   
                                          12/27/1999   
                                          12/27/1999   
                                          12/27/1999   
                                          12/27/1999   
                                          12/27/1999   
                                          12/27/1999   
                                          12/27/1999   
                                          12/27/1999   
                                          12/26/1999   
                                          12/26/1999   
                                          12/26/1999   
                                          12/26/1999   
                                          12/26/1999   
                                          12/26/1999   
                                          12/26/1999   
                                          12/25/1999   
                                          12/25/1999   
                                          12/25/1999   
                                          12/25/1999   
                                          12/25/1999   
                                          12/25/1999   
                                          12/25/1999   
                                          12/25/1999   
                                          12/24/1999   
                                          12/24/1999   
                                          12/24/1999   
                                          12/24/1999   
                                          12/24/1999   
                                          12/24/1999   
                                          12/24/1999   
                                          12/24/1999   
                                          12/23/1999   
                                          12/23/1999   
                                          12/23/1999   
                                          12/23/1999   
                                          12/23/1999   
                                          12/23/1999   
                                          12/22/1999   
                                          12/22/1999   
                